setup environment

In [ ]:
import pandas as pd
import numpy as np

read csv file
extract concepts and students

In [ ]:
fname = r''
df = pd.read_csv(fname)
concepts = list(df.columns[1:])
students = list(df["student"])
id = df.index.tolist()
tot_stud = len(id)

inital test breakdown: find students who lost points in each category
breakdown percentage: % of total students who lost points in each category

In [ ]:

test_bd = {}
for idx in concepts:
    test_bd[idx] = np.array([])
    for i in id:
        if f'{df[idx][i]}' != str(0):
            test_bd[idx] = np.append(test_bd[idx], f'{df["student"][i]}').tolist()

test_bd_pct = {}
for idx in test_bd:
    test_bd_pct[idx] = round(len(test_bd[idx])/tot_stud, 2)
    

topic-based breakdown: find correlations within each topic

In [ ]:
topic_bd = []
topic_bd_pct = {}
for idx in concepts:
    s = len(test_bd[idx])
    temp = [x for x in concepts if x != idx]
    for i in range(len(temp)):
        corr = [v for v in test_bd[idx] if v in test_bd[temp[i]]]
        n = f'{idx}_{temp[i]}'
        topic_bd.append({n:corr})
        topic_bd_pct[n] = round(len(corr)/s, 2)


displaying visualizations
